In [19]:
from azureml.data.dataset_factory import TabularDatasetFactory
from azureml.core import Dataset, Datastore

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

### YOUR CODE HERE ###
data_path='https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv'
ds =Dataset.Tabular.from_delimited_files(path=data_path)



In [20]:
import pandas as pd
def clean_data(data):
    # Dict for cleaning data
    months = {"jan":1, "feb":2, "mar":3, "apr":4, "may":5, "jun":6, "jul":7, "aug":8, "sep":9, "oct":10, "nov":11, "dec":12}
    weekdays = {"mon":1, "tue":2, "wed":3, "thu":4, "fri":5, "sat":6, "sun":7}

    # Clean and one hot encode data
    print("data to pandas")
    x_df = data.to_pandas_dataframe().dropna()
    print("one hot encode")
    jobs = pd.get_dummies(x_df.job, prefix="job")
    print("job dropped")
    x_df.drop("job", inplace=True, axis=1)
    print("adding jobs")
    x_df = x_df.join(jobs)
    print("more cleaning")
    x_df["marital"] = x_df.marital.apply(lambda s: 1 if s == "married" else 0)
    x_df["default"] = x_df.default.apply(lambda s: 1 if s == "yes" else 0)
    x_df["housing"] = x_df.housing.apply(lambda s: 1 if s == "yes" else 0)
    x_df["loan"] = x_df.loan.apply(lambda s: 1 if s == "yes" else 0)
    print("one hot contact")
    contact = pd.get_dummies(x_df.contact, prefix="contact")
    x_df.drop("contact", inplace=True, axis=1)
    x_df = x_df.join(contact)
    print("one hot education")
    education = pd.get_dummies(x_df.education, prefix="education")
    x_df.drop("education", inplace=True, axis=1)
    x_df = x_df.join(education)
    print("maping")
    x_df["month"] = x_df.month.map(months)
    x_df["day_of_week"] = x_df.day_of_week.map(weekdays)
    x_df["poutcome"] = x_df.poutcome.apply(lambda s: 1 if s == "success" else 0)
    print("end")
    y_df = x_df.pop("y").apply(lambda s: 1 if s == "yes" else 0)
    return x_df,y_df

In [24]:
df=ds.to_pandas_dataframe()
df.head()

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,...,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y
0,57,technician,married,high.school,no,no,yes,cellular,may,mon,...,1,999,1,failure,-1.8,92.893,-46.2,1.299,5099.1,no
1,55,unknown,married,unknown,unknown,yes,no,telephone,may,thu,...,2,999,0,nonexistent,1.1,93.994,-36.4,4.860,5191.0,no
2,33,blue-collar,married,basic.9y,no,no,no,cellular,may,fri,...,1,999,1,failure,-1.8,92.893,-46.2,1.313,5099.1,no
3,36,admin.,married,high.school,no,no,no,telephone,jun,fri,...,4,999,0,nonexistent,1.4,94.465,-41.8,4.967,5228.1,no
4,27,housemaid,married,high.school,no,yes,no,cellular,jul,fri,...,2,999,0,nonexistent,1.4,93.918,-42.7,4.963,5228.1,no


In [21]:
# Use the clean_data function to clean your data.
### YOUR DATA OBJECT HERE ###)
x, y = clean_data(ds)

data to pandas
one hot encode
job dropped
adding jobs
more cleaning
one hot contact
one hot education
maping
end


In [29]:
training_data=x.join(y)
training_data.head()

,age,marital,default,housing,loan,month,day_of_week,duration,campaign,pdays,...,contact_telephone,education_basic.4y,education_basic.6y,education_basic.9y,education_high.school,education_illiterate,education_professional.course,education_university.degree,education_unknown,y
0,57,1,0,0,1,5,1,371,1,999,...,0,0,0,0,1,0,0,0,0,0
1,55,1,0,1,0,5,4,285,2,999,...,1,0,0,0,0,0,0,0,1,0
2,33,1,0,0,0,5,5,52,1,999,...,0,0,0,1,0,0,0,0,0,0
3,36,1,0,0,0,6,5,355,4,999,...,1,0,0,0,1,0,0,0,0,0
4,27,1,0,1,0,7,5,189,2,999,...,0,0,0,0,1,0,0,0,0,0


In [30]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    task="classification",
    primary_metric="accuracy",
    training_data=training_data,
    label_column_name="y",
    n_cross_validations=5)

In [31]:
from azureml.core import Workspace, Experiment
#change myworspace name accordingly
ws = Workspace.get(name="myworkspace")
# Submit your automl run
experiment = Experiment(ws, "automl_test_experiment")
run = experiment.submit(config=automl_config, show_output=True)

Running on local machine
Parent Run ID: AutoML_20bf90c1-ff17-4ab7-b84d-c5ec3b9955a3

Current status: DatasetEvaluation. Gathering dataset statistics.
Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetFeaturization. Beginning to fit featurizers and featurize the dataset.
Current status: DatasetFeaturizationCompleted. Completed fit featurizers and featurizing the dataset.
Current status: DatasetBalancing. Performing class balancing sweeping
Current status: DatasetCrossValidationSplit. Generating individually featurized CV splits.

****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       ALERTED
DESCRIPTION:  To decrease model bias, please cancel the current run and fix balancing problem.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData
DETAILS:      Imbalanced data can lead to a falsely p

In [34]:
# Retrieve and save your best automl model.

### YOUR CODE HERE ###
#best_run = run.
#best_run_metrics = best_run.get_
#parameter_values = best_run.get_details()['runDefinition']['Arguments']
best_run, model = run.get_output()
estimator = model.steps[-1]

In [35]:
best_run.get_details()

{'runId': 'AutoML_20bf90c1-ff17-4ab7-b84d-c5ec3b9955a3_33',
 'status': 'Completed',
 'startTimeUtc': '2020-12-18T11:01:19.20084Z',
 'endTimeUtc': '2020-12-18T11:02:40.403796Z',
 'properties': {'runTemplate': 'automl_child',
  'pipeline_id': '__AutoML_Ensemble__',
  'pipeline_spec': '{"pipeline_id":"__AutoML_Ensemble__","objects":[{"module":"azureml.train.automl.ensemble","class_name":"Ensemble","spec_class":"sklearn","param_args":[],"param_kwargs":{"automl_settings":"{\'task_type\':\'classification\',\'primary_metric\':\'accuracy\',\'verbosity\':20,\'ensemble_iterations\':15,\'is_timeseries\':False,\'name\':\'automl_test_experiment\',\'compute_target\':\'local\',\'subscription_id\':\'26fa4a48-44da-4329-be20-b2ed37dcaadc\',\'region\':\'southcentralus\',\'spark_service\':None}","ensemble_run_id":"AutoML_20bf90c1-ff17-4ab7-b84d-c5ec3b9955a3_33","experiment_name":null,"workspace_name":"myworkspace","subscription_id":"26fa4a48-44da-4329-be20-b2ed37dcaadc","resource_group_name":"sdk"}}]}',
 

In [36]:
best_run_metrics = best_run.get_metrics()
parameter_values = best_run.get_details()['runDefinition']['Arguments']

print('Best Run Id: ', best_run.id)
print('\n Accuracy:', best_run_metrics['accuracy'])
#print('\n C :',parameter_values[3])
#print('\n :',parameter_values[5])
#print('\n batch size:',parameter_values[7])

KeyError: 'runDefinition'